https://bokeh.pydata.org/en/latest/docs/reference/palettes.html

In [243]:
from bokeh.io import (show, output_file)
from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    LinearColorMapper
)
from bokeh.palettes import OrRd9 as palette
from bokeh.palettes import BuGn9 as palette_fraud
from bokeh.palettes import BuGn9 as palette_theft


palette.reverse()
palette_fraud.reverse()
palette_theft.reverse()
from bokeh.plotting import figure
#bokeh.sampledata.download()
from bokeh.sampledata.us_counties import data as counties
from bokeh.sampledata.unemployment import data as unemployment

import pandas as pd
import numpy as np

from rudatasciproject1 import geoTime

## Try using US zip code Mapping with Bokeh

In [14]:
df_zipcode_geo = pd.read_csv("US_ZIP_codes.csv")
df_counties = df_zipcode_geo[(df_zipcode_geo['STATE']=="CA") & ([len(str(item))>4 for item in df_zipcode_geo['ZIP']])]

df_counties_LA = df_counties[df_counties['PONAME']=='LOS ANGELES']

df_LA_key = pd.read_csv("LA_Type_Key.csv")
df_LA_key = df_LA_key[df_LA_key["Description"]!="ABORTION/ILLEGAL"]


##### Add a column for the higher crime category

In [252]:
year = "2016"
filename = year+"_LA_Crimes.csv"
crimeType = "Theft"

df_LA = pd.read_csv(filename)
df_LA_ds = df_LA[["Zip code","Crime Code Description","DR Number"]]

df_LA_aggr = df_LA_ds.set_index("Crime Code Description").join(df_LA_key.set_index("Description"),how="inner")
df_LA_aggr = df_LA_aggr.reset_index().rename(columns={"index":"Description"})
df_LA_aggr = df_LA_aggr.set_index("Zip code").join(df_counties_LA.set_index("ZIP")[['geometry']],how="inner")
df_LA_aggr = df_LA_aggr.reset_index().rename(columns={"index":"Zip code"})

df_LA_zipvsgeo = pd.DataFrame(df_LA_aggr.set_index("Zip code")["geometry"].drop_duplicates())
df_LA_zipvscrime = df_LA_aggr.groupby(["Zip code","Type"]).count()["Description"].unstack()

df_data = df_LA_zipvsgeo.join(df_LA_zipvscrime,how="inner").reset_index()
df_data = df_data[['geometry','Zip code',crimeType]].dropna()

zip_xs = [geoTime.getArrayFromGeokml(zipCodeGeo)[0] for zipCodeGeo in df_data['geometry']]
ZipNames = [str(zipname) for zipname in df_data['Zip code']]
zip_ys = [geoTime.getArrayFromGeokml(zipCodeGeo)[1] for zipCodeGeo in df_data['geometry']]
rates = list(df_data[crimeType])


color_mapper = LinearColorMapper(palette=palette_theft)

source = ColumnDataSource(data=dict(
    x=zip_xs,
    y=zip_ys,
    name=ZipNames,
    rate=rates,
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title=crimeType, tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)

p.grid.grid_line_color = None


p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    (crimeType, "@rate"),
]

output_file(crimeType+"_"+year+"_LA.html")
show(p)

In [259]:
df_zipcode_geo = pd.read_csv("US_ZIP_codes.csv")
df_counties = df_zipcode_geo[(df_zipcode_geo['STATE']=="NY") & ([len(str(item))>4 for item in df_zipcode_geo['ZIP']])]

df_NYC_key = pd.read_csv("NYC_Type_Key.csv")

year = "2016"
filename = year+"_NYC_Crimes.csv"
crimeType = "Theft"

df_NYC = pd.read_csv(filename)
df_NYC_ds = df_NYC[["Zip code","OFNS_DESC","CMPLNT_NUM"]]


df_NYC_aggr = df_NYC_ds.set_index("OFNS_DESC").join(df_NYC_key.set_index("Description"),how="inner")
df_NYC_aggr = df_NYC_aggr.reset_index().rename(columns={"index":"Description"})
df_NYC_aggr = df_NYC_aggr.set_index("Zip code").join(df_counties.set_index("ZIP")[['geometry']],how="inner")
df_NYC_aggr = df_NYC_aggr.reset_index().rename(columns={"index":"Zip code"})


df_NYC_zipvsgeo = pd.DataFrame(df_NYC_aggr.set_index("Zip code")["geometry"].drop_duplicates())
df_NYC_zipvscrime = df_NYC_aggr.groupby(["Zip code","Type"]).count()["Description"].unstack()

df_data = df_NYC_zipvsgeo.join(df_NYC_zipvscrime,how="inner").reset_index()
df_data = df_data[['geometry','Zip code',crimeType]].dropna()

zip_xs = [geoTime.getArrayFromGeokml(zipCodeGeo)[0] for zipCodeGeo in df_data['geometry']]
ZipNames = [str(zipname) for zipname in df_data['Zip code']]
zip_ys = [geoTime.getArrayFromGeokml(zipCodeGeo)[1] for zipCodeGeo in df_data['geometry']]
rates = list(df_data[crimeType])


color_mapper = LinearColorMapper(palette=palette_theft)

source = ColumnDataSource(data=dict(
    x=zip_xs,
    y=zip_ys,
    name=ZipNames,
    rate=rates,
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title=crimeType, tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)

p.grid.grid_line_color = None


p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    (crimeType, "@rate"),
]

output_file(crimeType+"_"+year+"_NYC.html")
show(p)


In [266]:
df_zipcode_geo = pd.read_csv("US_ZIP_codes.csv")
df_counties = df_zipcode_geo[(df_zipcode_geo['STATE']=="IL") & ([len(str(item))>4 for item in df_zipcode_geo['ZIP']])]

df_CHI_key = pd.read_csv("CHI_Type_Key.csv")

year = "2016"
filename = year+"_CHI_Crimes.csv"
crimeType = 'Theft'

df_CHI = pd.read_csv(filename)



df_CHI_ds = df_CHI[["Zip code","Primary Type","Case Number"]]


df_CHI_aggr = df_CHI_ds.set_index("Primary Type").join(df_CHI_key.set_index("Description"),how="inner")
df_CHI_aggr = df_CHI_aggr.reset_index().rename(columns={"index":"Description"})
df_CHI_aggr = df_CHI_aggr.set_index("Zip code").join(df_counties.set_index("ZIP")[['geometry']],how="inner")
df_CHI_aggr = df_CHI_aggr.reset_index().rename(columns={"index":"Zip code"})


df_CHI_zipvsgeo = pd.DataFrame(df_CHI_aggr.set_index("Zip code")["geometry"].drop_duplicates())
df_CHI_zipvscrime = df_CHI_aggr.groupby(["Zip code","Type"]).count()["Description"].unstack()

df_data = df_CHI_zipvsgeo.join(df_CHI_zipvscrime,how="inner").reset_index()
df_data = df_data[['geometry','Zip code',crimeType]].dropna()

zip_xs = [geoTime.getArrayFromGeokml(zipCodeGeo)[0] for zipCodeGeo in df_data['geometry']]
ZipNames = [str(zipname) for zipname in df_data['Zip code']]
zip_ys = [geoTime.getArrayFromGeokml(zipCodeGeo)[1] for zipCodeGeo in df_data['geometry']]
rates = list(df_data[crimeType])


color_mapper = LinearColorMapper(palette=palette_theft)

source = ColumnDataSource(data=dict(
    x=zip_xs,
    y=zip_ys,
    name=ZipNames,
    rate=rates,
))

TOOLS = "pan,wheel_zoom,reset,hover,save"

p = figure(
    title=crimeType, tools=TOOLS,
    x_axis_location=None, y_axis_location=None
)

p.grid.grid_line_color = None


p.patches('x', 'y', source=source,
          fill_color={'field': 'rate', 'transform': color_mapper},
          fill_alpha=0.7, line_color="white", line_width=0.5)

hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [
    ("Name", "@name"),
    (crimeType, "@rate"),
]

output_file(crimeType+"_"+year+"_CHI.html")
show(p)

